https://github.com/martinarjovsky/WassersteinGAN

https://vincentherrmann.github.io/blog/wasserstein/

https://github.com/martinarjovsky/WassersteinGAN/issues/2

https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py

# To do:

* Clip and then intorudce logs (should go to -17,9)
* Overfit small dataset
* Add confidence intervals to (mean and std)
* Upload redshift 5
* Fix sns denisty plots
* Check for overfitting
* Play around with optimizers
* Clean code, integrate transformations
* Revise and change architecture (less convs, polling, FC), add convolution after ReLU in generator
* Mean and standard errors of power spectrums
* Increase channels
* Bigger cubes with no padding
* Use other standarizations of data
* Gradient penalty instead of weight clamping
* Locally connected layers
* Weight initialization
* Mini batch discrimination

In [1]:
from __future__ import print_function
#import argparse
import os
import random
import numpy as np

#import torch

import torch.multiprocessing as mp
mp.set_start_method('spawn')

import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable, grad

import matplotlib.pyplot as plt
%matplotlib inline

import h5py

import itertools
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import matplotlib as mpl
import timeit

import seaborn as sns

ngpu = 1   

from power_spectrum_utils import power_spectrum_np
from wgan_utils import check_coords #, define_test
#from data_utils import get_max_cube, get_min_cube, get_mean_cube, get_stddev_cube
from plot_utils import plot_loss, plot_means

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#mean_5=get_mean_cube('../../../../../','5.0')
#std_5=get_stddev_cube('../../../../../','5.0',mean_5)
#
mean_5=14592.24
std_5=922711.56
max_5=4376932000
mask_value=10**2

#Bootstrap
mean_l5 = 2.7784111
std_l5 = 1.5777067
max_l5 =22.199614

In [3]:
#get_ipython().run_line_magic('run', '../mmd_gan/utils/power_spectrum_utils.py')
#get_ipython().run_line_magic('run', '../mmd_gan/utils/plot_utils.py')

def moving_average(data_set, periods=3):
    weights = np.ones(periods) / periods
    return np.convolve(data_set, weights, mode='valid')



def plot_power_spec(real_cube,        # should be inverse_transformed
                    generated_cube,   # should be inverse_transformed
                    raw_cube_mean,
                    s_size,
                    threads=1, 
                    MAS="CIC", 
                    axis=0, 
                    BoxSize=75.0/2048*64):
    """Takes as input;
    - Real cube: (batch_size x 1 x n x n x n) torch cuda FloatTensor,
    - Generated copy: (batch_size x 1 x n x n x n) torch cuda FloatTensor,
    - constant assignments: threads, MAS, axis, BoxSize.
    
    Returns;
    - Power spectrum plots of both cubes
    in the same figure.
    """
    real_cube = real_cube.reshape(-1,
                                  1,
                                  real_cube.shape[2],
                                  real_cube.shape[2],
                                  real_cube.shape[2])
    generated_cube = generated_cube.reshape(-1,
                                            1,
                                            generated_cube.shape[2],
                                            generated_cube.shape[2],
                                            generated_cube.shape[2])
    
    #print("number of samples of real and generated cubes = " + str(real_cube.shape[0]))

    plt.figure(figsize=(14,8))
    
    for cube_no in range(real_cube.shape[0]):
        
        delta_real_cube = real_cube[cube_no][0]
        delta_gen_cube = generated_cube[cube_no][0]
        
        k_real ,Pk_real = power_spectrum_np(cube = delta_real_cube, mean_raw_cube = raw_cube_mean, SubBoxSize=s_sample)
        k_gen,  Pk_gen  = power_spectrum_np(cube = delta_gen_cube,  mean_raw_cube = raw_cube_mean, SubBoxSize=s_sample)
    
        plt.plot(np.log(k_real), np.log(Pk_real), color="b", alpha = 0.8, label="Real", linewidth = 2)
        plt.plot(np.log(k_gen), np.log(Pk_gen), color="r", alpha = 0.8, label="Generated", linewidth = 2)
        
        plt.rcParams["font.size"] = 16
        plt.title("Power Spectrum Comparison - (Blue: Real, Red: Generated)")
        plt.xlabel('logk')
        plt.ylabel('log(Pk.k3D)')
        #plt.xlim(0, 3)
#         plt.legend()
    
    #if save_plot:
     #   plt.savefig(redshift_fig_folder + 'powerspectrum_' + str(t) + '.png', 
      #              bbox_inches='tight')
   
    plt.show()
    #return np.log10(k_real), np.log10(k_gen)

In [4]:
class HydrogenDataset(Dataset):
    """Hydrogen Dataset"""

    def __init__(self, h5_file, s_test, s_sample, nsamples,transform, rotate):
        """
        Args:
            h5_file (string): name of the h5 file with 32 sampled cubes.
            root_dir (string): Directory with the .h5 file.
        """
        #file_size = os.path.getsize(root_dir + h5_file) / 1e6 # in MBs
        #print("The whole file size is " + str(int(file_size)) + " MBs")
        # self.subcubes = h5py.File('../data/sample_32.h5', 'r')
        self.s_test = s_test
        self.s_sample = s_sample
        self.t_coords = {'x': [0, 1023], 'y': [0, 1023], 'z': [0, 1023]} # define_test(self.s_test,self.s_sample)
        self.nsamples = nsamples
        self.redshift = redshift
        self.transform=transform
        self.rotate=rotate
        self.file=h5_file

    def __len__(self):
        # Function called when len(self) is executed
        return self.nsamples

    def __getitem__(self, idx):
        """
        This can be implemented in such a way that the whole h5 file read 
        using h5py.File() and get_sample() function is called to return
        a random subcube. This won't increase memory usage because the
        subcubes will be read in the same way and only the batch will
        be read into memory.
        
        Here we have implemented it so that it can be used with data
        generated by get_sample() function.
        
        The output of this function is one subcube with the dimensions
        specified by get_sample() implementation.
        """
        
        sample = get_samples(file = self.file,
                            s_sample = self.s_sample,
                             nsamples = 1,
                             test_coords = self.t_coords)
        
        if self.rotate==True:
            t=random.randint(1,4)
            if t==4:
                sample=sample
            else:
                axis=random.choice([(0,1), (0,2), (1,0), (1,2), (2,0), (2,1) ])
                sample=np.rot90(sample, t, axis)
        
        #sample=np.array(sample)
        if self.transform=='max':
            #print(sample.shape)
            sample = sample / self.max_cube
        if self.transform=='normalize':
            #sample = (sample-mean_5)/std_5
            sample = (sample-mean_5)/std_5
            #sample[sample <10**2]==0
            #sample[sample > 3*std_5]==3*std_5
            #sample=np.log(sample+1)
        if self.transform=='clip_log':
            sample[sample <10**2]==0
            sample[sample > 3*std_5]==3*std_5
            sample=np.log(sample+1)
        
        if self.transform=='log_max':
            #sample[sample==0]=0.000000001
            sample=np.log(sample+1) / max_l5
            
        if self.transform=='log':
            sample= np.log(sample+1)
        #print(np.array(sample).shape)
    
        sample=sample.reshape((1, self.s_sample, self.s_sample, self.s_sample))

        return torch.tensor(sample)

def get_samples(file, s_sample, nsamples, test_coords):
    #n is size of minibatch, get valid samples (not intersecting with test_coords)
    sample_list=[]
    m=2048-s_sample
    for n in range(nsamples):
        #print("Sample No = " + str(n + 1) + " / " + str(nsamples))
        sample_valid=False
        while sample_valid==False:
            x = random.randint(0,m)
            y = random.randint(0,m)
            z = random.randint(0,m)
            sample_coords = {'x':[x,x+s_sample], 
                             'y':[y,y+s_sample], 
                             'z':[z,z+s_sample]}
            
            sample_valid = check_coords(test_coords, sample_coords)
        
        sample_list.append(sample_coords)
    
    #Load cube and get samples and convert them to np.arrays
    sample_array=[]
    #f file has to be opened outisde the function
    for c in sample_list:
        a = f[c['x'][0]:c['x'][1],
              c['y'][0]:c['y'][1],
              c['z'][0]:c['z'][1]]
        
        sample_array.append(np.array(a))
    
    return np.array(sample_array)

In [5]:
def visualize2d(real, fake, log=False, save=''):
    #min_=-.5
    min_= 0
    max_= .3
    #max_=(max_l5-mean_l5) / std_l5
    
    cols=8
    rows=2
    
    fig, axes = plt.subplots(nrows=2, ncols=cols, figsize=(16,4))
    
    for ax, row in zip(axes[:,0], ['Generated', 'Real']):
        ax.set_ylabel(row, rotation=90, fontsize=16)
    
    mf, mr= 0,0
    for ax in axes.flat:
        #Plot only half of the mini-batch
        if mf<cols:
            if log==True:
                im = ax.imshow(np.log(fake[mf][0].mean(axis=2)), aspect='equal', #cmap='Blues',
                       interpolation=None, vmin=min_, vmax=max_)
            else:
                im = ax.imshow((fake[mf][0].mean(axis=2)), aspect='equal', #cmap='Blues',
                       interpolation=None, vmin=min_, vmax=max_)
            mf+=1
        else:
            if log==True:
                im = ax.imshow(np.log(real[mr][0].mean(axis=2)), aspect='equal', #cmap='Blues',
                       interpolation=None, vmin=min_, vmax=max_)
            else:
                im = ax.imshow((real[mr][0].mean(axis=2)), aspect='equal', #cmap='Blues',
                       interpolation=None, vmin=min_, vmax=max_)
                
            mr+=1
        ax.set_xticks([])
        ax.set_yticks([])
    #fig.subplots_adjust(right=.8)
    #cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    #fig.colorbar(ax, cax=cbar_ax)
    fig.tight_layout() 
    if save!='':
        fig.savefig(save+'samples.png', dpi=1000)
           
    plt.show()  
    
def hist_plot(noise, real, log_plot, redshift_fig_folder) :
    """
    Args:
        real(): real data
        epoch(integer): epoch number
        file_name(string): name of the file
        hd (integer) : if 0 it's a histogram, if 1 it's a pdf
        
    """
    
    plt.figure(figsize = (20,10))
    plot_min = min(float(noise.min()), float(real.min()))
    plot_max = max(float(noise.max()), float(real.max()))
    plt.xlim(plot_min,plot_max)
    
    bins = np.linspace(plot_min,plot_max,400)
    
    real_label = "Real Subcube - Only Nonzero"
    noise_label = "Noise Subcube - Only Nonzero"
    
    for m in range(noise.shape[0]):
        if m<=8:
            plt.hist(real[m][0].flatten(), bins = bins, color = "b" , log = log_plot, alpha = 0.3, label = real_label, normed=True)
            plt.hist(noise[m][0].flatten(), bins = bins, color = "r" , log = log_plot, alpha= 0.3, label = noise_label, normed=True)

    #plt.legend()
    #plt.savefig(redshift_fig_folder + file_name, bbox_inches='tight')   
    plt.title('Blue: Real/ Red: Generated', fontsize=16)
    plt.show()

In [6]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
'''

#https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py
def calc_gradient_penalty(real_data, fake_data, lambda_):
    # print "real_data: ", real_data.size(), fake_data.size()
    BATCH_SIZE=real_data.size()[0]
    
    alpha = torch.rand(BATCH_SIZE, 1, 1)
    alpha = alpha.expand(BATCH_SIZE, real_data.nelement()//BATCH_SIZE).contiguous().view(BATCH_SIZE, 1,
                                                                                         s_sample, s_sample,s_sample)
    alpha = alpha.cuda() if cuda else alpha

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    if cuda:
        interpolates = interpolates.cuda()
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda() if cuda else torch.ones(
                                  disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_
    return gradient_penalty

'''

##https://github.com/EmilienDupont/wgan-gp/blob/master/training.py
def _gradient_penalty(real_data, generated_data, gp_weight):
        batch_size = real_data.size()[0]

        # Calculate interpolation
        alpha = torch.rand(batch_size, 1, 1, 1, 1)
        alpha = alpha.expand_as(real_data)
        if cuda:
            alpha = alpha.cuda()
        interpolated = alpha * real_data.data + (1 - alpha) * generated_data.data
        interpolated = Variable(interpolated, requires_grad=True)
        if cuda:
            interpolated = interpolated.cuda()

        # Calculate probability of interpolated examples
        prob_interpolated = netD(interpolated)

        # Calculate gradients of probabilities with respect to examples
        gradients = grad(outputs=prob_interpolated, inputs=interpolated,
                               grad_outputs=torch.ones(prob_interpolated.size()).cuda() if cuda else torch.ones(
                               prob_interpolated.size()),
                               create_graph=True, retain_graph=True)[0]

        # Gradients have shape (batch_size, num_channels, img_width, img_height),
        # so flatten to easily take norm per example in batch
        gradients = gradients.view(batch_size, -1)
        #self.losses['gradient_norm'].append(gradients.norm(2, dim=1).mean().data[0])

        # Derivatives of the gradient close to 0 can cause problems because of
        # the square root, so manually calculate norm and add epsilon
        gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

        # Return gradient penalty
        return gp_weight * ((gradients_norm - 1) ** 2).mean()

In [7]:
class DCGAN_G(nn.Module):
    def __init__(self, isize, nz, nc, ngf, ngpu, n_extra_layers):
        super(DCGAN_G, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        #main.add_module('initial_fc'.format(nz, nz),
        #               nn.ConvTranspose2d(nz, nz, 1, 1, 0),)
                   
       #     nn.LeakyReLU(0.2, inplace=True),
        # input is Z, going into a convolution
        main.add_module('initial_{0}-{1}_convt'.format(nz, cngf),
                        nn.ConvTranspose3d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial_{0}_batchnorm'.format(cngf),
                        nn.BatchNorm3d(cngf))
        main.add_module('initial_{0}_relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid_{0}-{1}_convt'.format(cngf, cngf//2),
                            nn.ConvTranspose3d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid_{0}_batchnorm'.format(cngf//2),
                            nn.BatchNorm3d(cngf//2))
            main.add_module('pyramid_{0}_relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}_{1}_conv'.format(t, cngf),
                            nn.Conv3d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}_{1}_batchnorm'.format(t, cngf),
                            nn.BatchNorm3d(cngf))
            main.add_module('extra-layers-{0}_{1}_relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final_{0}-{1}_convt'.format(cngf, nc),
                        nn.ConvTranspose3d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final_{0}_tanh'.format(nc),
                        nn.Sigmoid())
        self.main = main

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
        return output 

In [8]:
class DCGAN_D(nn.Module):
    def __init__(self, isize, nz, nc, ndf, ngpu, n_extra_layers):
        super(DCGAN_D, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial_conv_{0}-{1}'.format(nc, ndf),
                        nn.Conv3d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial_relu_{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}_{1}_conv'.format(t, cndf),
                            nn.Conv3d(cndf, cndf, 3, 1, 1, bias=False))
            #main.add_module('extra-layers-{0}_{1}_batchnorm'.format(t, cndf),
            #                nn.BatchNorm3d(cndf))
            main.add_module('extra-layers-{0}_{1}_relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid_{0}-{1}_conv'.format(in_feat, out_feat),
                            nn.Conv3d(in_feat, out_feat, 4, 2, 1, bias=False))
           # main.add_module('pyramid_{0}_batchnorm'.format(out_feat),
            #                nn.BatchNorm3d(out_feat))
            main.add_module('pyramid_{0}_relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        main.add_module('final_{0}-{1}_conv'.format(cndf, 1),
                        nn.Conv3d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
            
        output = output.mean(0)
        return output.view(1)

In [ ]:
#New training scripts
redshift ='5.0'
cuda = True
n_samples = 64000
s_sample = 64
batchSize = 8
n_extra_layers_G = 0
n_extra_layers_D = 0

nz = 100
nc = 1
ngf = 128
ndf = ngf

niter = 30 #epochs
Diters_init = 5

manualSeed = 1   # 'manual seed'

#.0002
#Default: 0.00005
lrD = 0.00004
lrG = 0.00004

#Clamp weights or use GP
#clamp_lower = -.012  #Default .01
#clamp_upper = .012
lambda_ = 10

device = torch.device("cuda:0" if cuda else "cpu")
experiment='first_iteration/'

DCGAN=True
#DC
if DCGAN == True:
    netG = DCGAN_G(s_sample, nz, nc, ngf, ngpu, n_extra_layers_G)
    netD = DCGAN_D(s_sample, nz, nc, ngf, ngpu, n_extra_layers_D)
    #netG = DCGAN_G(ngpu, 4, 1, False)
    #netD = DCGAN_D(ngpu, 4, 1, False)
#MLP
else:
    netG = MLP_G(s_sample, nz, nc, ngf, ngpu)
    netD = MLP_D(s_sample, nz, nc, ndf, ngpu)

#netG_arg = "first_iteration/saved/DCGAN/netG_epoch_63.pth"        # "path to netG (to continue training) "
#netD_arg = "first_iteration/saved//DCGAN/netD_epoch_63.pth"  
netG_arg = ''
netD_arg = ''

netG.apply(weights_init)
netD.apply(weights_init)


if netG_arg != '':
    netG.load_state_dict(torch.load(netG_arg))

if netD_arg != '':
    netD.load_state_dict(torch.load(netD_arg))


datapath='../../../../../'
f = h5py.File(datapath+'fields_z='+redshift+'.hdf5', 'r')
f=f['delta_HI']
    
dataset = HydrogenDataset( h5_file=0,
                            s_test = 1024, 
                            s_sample = s_sample, 
                            nsamples = n_samples, 
                            transform='log_max',
                            rotate=True)

workers=0
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size = batchSize,
                                         shuffle=False, 
                                         #num_workers=int(opt.workers)
                                         num_workers=int(workers),
                                        drop_last = True)

print(netG)
print(netD)

DCGAN_G(
  (main): Sequential(
    (initial_100-1024_convt): ConvTranspose3d(100, 1024, kernel_size=(4, 4, 4), stride=(1, 1, 1), bias=False)
    (initial_1024_batchnorm): BatchNorm3d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (initial_1024_relu): ReLU(inplace)
    (pyramid_1024-512_convt): ConvTranspose3d(1024, 512, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
    (pyramid_512_batchnorm): BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_512_relu): ReLU(inplace)
    (pyramid_512-256_convt): ConvTranspose3d(512, 256, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
    (pyramid_256_batchnorm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid_256_relu): ReLU(inplace)
    (pyramid_256-128_convt): ConvTranspose3d(256, 128, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
    (pyramid_128_batchnorm): 

In [ ]:
# setup optimizer
if lambda_ is None:
    optimizerD = optim.RMSprop(netD.parameters(), lr = lrD)
    optimizerG = optim.RMSprop(netG.parameters(), lr = lrG)
else:
    optimizerD = optim.Adam(netD.parameters(), lr=lrD, betas=(0.5, 0.9))
    optimizerG = optim.Adam(netG.parameters(), lr=lrG, betas=(0.5, 0.9))

wass_loss=[]
errG_l=[]
errD_real_l=[]
errD_fake_l=[]

input = torch.FloatTensor(batchSize, 1, s_sample, s_sample, s_sample)
noise = torch.FloatTensor(batchSize, nz,1 , 1, 1,  device=device).normal_(0,1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1, 1 ).normal_(0, 1)
#fixed_noise = torch.FloatTensor(1, nz, 1, 1,1).normal_(0, 1)

one = torch.FloatTensor([1])
#one = torch.tensor(1, dtype=torch.float)
mone = one * -1

#torch.cuda.empty_cache()
if cuda==True:
    netD.cuda()
    netG.cuda()
    input = input.cuda()
    one, mone = one.cuda(), mone.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()


gen_iterations = 0
for epoch in range(niter):
    
    data_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): # reset requires_grad
            p.requires_grad = True # they are set to False below in netG update
           
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            Diters = 20
        else:
            Diters = Diters_init
            
        j=0
        while j < Diters and i < len(dataloader):
            j += 1
            
            #########clamp parameters to a cube
            if lambda_ is None:
                for p in netD.parameters():
                    p.data.clamp_(clamp_lower, clamp_upper)
                
            data=data_iter.next()
            i+=1
            
            real_cpu= data
            netD.zero_grad()
          
            batch_size=real_cpu.size(0)
            if cuda==True:
                real_cpu=real_cpu.cuda()
        
            input.resize_as_(real_cpu).copy_(real_cpu)
            inputv = Variable(input)
            #print(inputv.size())   
            errD_real=netD(inputv)
            
            errD_real.backward(one)
                
            #Train with fake
            noise.resize_(batchSize, nz, 1, 1, 1).normal_(0, 1)

            with torch.no_grad():
                noisev = Variable(noise) # totally freeze netG
                
            fake = Variable(netG(noisev).data)

            inputv=fake
               
            errD_fake = netD(inputv)
            errD_fake.backward(mone)
            
            ## train with gradient penalty
            if lambda_ is not None:
                gradient_penalty = _gradient_penalty(real_cpu.data, fake.data, lambda_)
                gradient_penalty.backward()
                errD = errD_real - errD_fake + gradient_penalty
                wass_D =  errD_real - errD_fake
                
                optimizerD.step()
                wass_loss.append(float(wass_D.data[0]))
            else:
                errD =  errD_real - errD_fake
                optimizerD.step()
                wass_loss.append(float(errD.data[0]))
                
            errD_real_l.append(float(errD_real.data[0]))
            errD_fake_l.append(float(errD_fake.data[0]))
           
           
        ############################
        # (2) Update G network
        ###########################

        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        netG.zero_grad()
        # in case our last batch was the tail batch of the dataloader,
        # make sure we feed a full batch of noise
        noise.resize_(batchSize, nz, 1, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        #print('G:',fake.shape)
        errG = netD(fake)
        
        #print('errG: ', errG)
        #label = torch.full(size = (batch_size,), fill_value = real_label, device = device)
        errG.backward(one)
        optimizerG.step()
        
        gen_iterations += 1
        
        if lambda_ is not None:
            errD=wass_D
        
        print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, niter, i, len(dataloader), gen_iterations,
            errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))
        
        #wass_loss.append(float(errD.data[0]))
        errG_l.append(float(errG.data[0]))

        #if gen_iterations % 10==0  or (gen_iterations>5 and -float(errD.data[0])< min([-z for z in wass_loss])):
        if gen_iterations % 200 == 0:
            with torch.no_grad():
                fake = netG(Variable(fixed_noise))
                fake = np.array(fake)
                real_cpu = np.array(real_cpu)

            plot_power_spec((np.exp(fake * max_l5) -1), (np.exp(real_cpu * max_l5) -1), mean_5, s_sample)
            #visualize2d((np.array(real_cpu)), (np.array(fake)), save='')
            visualize2d(np.array(real_cpu), np.array(fake), log= False, save='')
            hist_plot(np.array(fake), np.array(real_cpu), log_plot=False, redshift_fig_folder='') 
            plot_loss(wass_loss,'Wasserstein loss', log_=False)
            
            #plot_loss(wass_loss,'Wasserstein loss', log_=False)
           
    # do checkpointing
    torch.save(netG.state_dict(), '{0}/netG_epoch_{1}.pth'.format(experiment, epoch))
    torch.save(netD.state_dict(), '{0}/netD_epoch_{1}.pth'.format(experiment, epoch))

[0/30][20/8000][1] Loss_D: -6234.222656 Loss_G: 10253.004883 Loss_D_real: 2140.118164 Loss_D_fake 8374.340820
[0/30][40/8000][2] Loss_D: -71655.343750 Loss_G: 99240.726562 Loss_D_real: 24794.707031 Loss_D_fake 96450.046875
[0/30][60/8000][3] Loss_D: -78862.484375 Loss_G: 108524.773438 Loss_D_real: 30429.722656 Loss_D_fake 109292.203125
[0/30][80/8000][4] Loss_D: -78524.609375 Loss_G: 111972.343750 Loss_D_real: 31087.251953 Loss_D_fake 109611.859375
[0/30][100/8000][5] Loss_D: -83420.539062 Loss_G: 113527.445312 Loss_D_real: 31173.689453 Loss_D_fake 114594.226562
[0/30][120/8000][6] Loss_D: -80328.515625 Loss_G: 112349.984375 Loss_D_real: 30418.375000 Loss_D_fake 110746.890625
[0/30][140/8000][7] Loss_D: -84146.914062 Loss_G: 114361.125000 Loss_D_real: 32731.302734 Loss_D_fake 116878.218750
[0/30][160/8000][8] Loss_D: -76770.890625 Loss_G: 107718.812500 Loss_D_real: 34148.093750 Loss_D_fake 110918.984375
[0/30][180/8000][9] Loss_D: -82542.734375 Loss_G: 116253.875000 Loss_D_real: 28336.

[0/30][740/8000][73] Loss_D: -6768.514160 Loss_G: 13646.152344 Loss_D_real: 7517.694824 Loss_D_fake 14286.208984
[0/30][745/8000][74] Loss_D: -7029.872559 Loss_G: 13904.264648 Loss_D_real: 7255.649902 Loss_D_fake 14285.522461
[0/30][750/8000][75] Loss_D: -7034.600098 Loss_G: 14415.845703 Loss_D_real: 6315.691895 Loss_D_fake 13350.291992
[0/30][755/8000][76] Loss_D: -6602.337402 Loss_G: 13444.418945 Loss_D_real: 6377.232910 Loss_D_fake 12979.570312
[0/30][760/8000][77] Loss_D: -6503.788574 Loss_G: 12674.130859 Loss_D_real: 6724.982910 Loss_D_fake 13228.771484
[0/30][765/8000][78] Loss_D: -6061.922852 Loss_G: 12199.586914 Loss_D_real: 6113.784180 Loss_D_fake 12175.707031
[0/30][770/8000][79] Loss_D: -5697.094727 Loss_G: 11576.278320 Loss_D_real: 6249.809570 Loss_D_fake 11946.904297
[0/30][775/8000][80] Loss_D: -5470.783203 Loss_G: 11347.610352 Loss_D_real: 5337.315430 Loss_D_fake 10808.098633
[0/30][780/8000][81] Loss_D: -5309.761230 Loss_G: 10913.257812 Loss_D_real: 5593.914551 Loss_D_f

In [ ]:
plot_loss(wass_loss,'Wasserstein loss', log_=False)
plot_loss(errG_l,'Generator loss', log_=False)
plot_loss(errD_real_l,'D real loss', log_=False)
plot_loss(errD_fake_l,'D fake loss', log_=False)

# Additional Architectures

In [ ]:
class DCGAN_G_(nn.Module):
    def __init__(self, ngpu,kernel,  dilation, bias):
        super(DCGAN_G, self).__init__()
        self.ngpu = ngpu
        self.kernel=kernel
        self.bias=bias
        self.dilation=dilation
        self.main = nn.Sequential(
            # input is Z,
            nn.ConvTranspose3d(in_channels=nz,
                               out_channels=ngf * 16,
                               kernel_size =self.kernel,
                               stride = 1,
                               padding = 0,
                               bias=False,
                              dilation=self.dilation),
            nn.BatchNorm3d(ngf * 16),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose3d(ngf * 16, ngf * 8, self.kernel, 2, 1, bias=self.bias, dilation=self.dilation),
            nn.BatchNorm3d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose3d(ngf * 8, ngf * 4, self.kernel, 2, 1, bias=self.bias,dilation=self.dilation),
            nn.BatchNorm3d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose3d(ngf * 4, ngf * 2, self.kernel, 2, 1, bias=self.bias,dilation=self.dilation),
            nn.BatchNorm3d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose3d(ngf * 2,     ngf, self.kernel, 2, 1, bias=self.bias,dilation=self.dilation),
            nn.BatchNorm3d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose3d(ngf, nc, self.kernel, 2, 1, bias=self.bias,dilation=self.dilation),
            nn.Tanh(),
            #nn.ReLU(True)
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        #print('G: ' ,output.size())
        return output

In [ ]:
#https://github.com/rimchang/3DGAN-Pytorch/blob/master/3D_GAN/model.py
'''Detect when wass loss is making a big jump and keep iterating the 
critic till the curve goes up again to roughly were it was before the jump.'''

class DCGAN_D_(nn.Module):
    def __init__(self, ngpu, kernel, dilation, bias):
        super(DCGAN_D, self).__init__()
        self.ngpu = ngpu
        self.kernel=kernel
        self.bias=bias
        self.dilation=dilation
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            # nc = number of channels
            
            nn.Conv3d(in_channels=nc, 
                      out_channels=ndf, 
                      kernel_size=self.kernel, # == 4( 4,4,4)
                      stride = 2, # == 2
                     padding=1,
                      dilation=self.dilation,
                      # == 1
                     # padding=0,
                      bias=self.bias),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 32 x 32
            nn.Conv3d(ndf, ndf * 2, self.kernel, 2, 1, bias=self.bias, dilation=self.dilation),
            nn.BatchNorm3d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*2) x 16 x 16
            nn.Conv3d(ndf * 2, ndf * 4, self.kernel, 2, 1, bias=self.bias, dilation=self.dilation),
            nn.BatchNorm3d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 8 x 8
            nn.Conv3d(ndf * 4, ndf * 8, self.kernel, 2, 1, bias=self.bias, dilation=self.dilation),
            nn.BatchNorm3d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 8 x 8
            nn.Conv3d(ndf * 8, ndf * 16, self.kernel, 2, 1, bias=self.bias, dilation=self.dilation),
            nn.BatchNorm3d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
           # nn.Conv3d(ndf * 16, ndf*32, 4, 2, 1, bias=False),
            #nn.BatchNorm3d(ndf * 32),
            #nn.LeakyReLU(0.2, inplace=True),
            
            #Final conv (D does not have ti output a probability)
            nn.Conv3d(ndf*16, 1, self.kernel, 1, 0, bias=self.bias, dilation=self.dilation)
        )
        #self.main = main

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main,  input, range(self.ngpu))
        else:
            output = self.main(input)
            
        #print('D0:',output.size())
        output = output.mean(0)
        #return output.view(-1, 1).squeeze(1)
        #print('D1:',output.size())
        return output.view(1)
    

# MLP

In [ ]:
class MLP_G(nn.Module):
    def __init__(self, isize, nz, nc, ngf, ngpu):
        super(MLP_G, self).__init__()
        self.ngpu = ngpu

        main = nn.Sequential(
            # Z goes into a linear of size: ngf
            nn.Linear(nz, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, ngf),
            nn.ReLU(True),
            nn.Linear(ngf, ngf),
            nn.ReLU(True),
            nn.Linear(ngf,  nc * isize * isize *isize),
            nn.ReLU(True)
        )
        self.main = main
        self.nc = nc
        self.isize = isize
        self.nz = nz

    def forward(self, input):
        input = input.view(input.size(0), input.size(1))
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output.view(output.size(0), self.nc, self.isize, self.isize, self.isize)

In [ ]:
class MLP_D(nn.Module):
    def __init__(self, isize, nz, nc, ndf, ngpu):
        super(MLP_D, self).__init__()
        self.ngpu = ngpu

        main = nn.Sequential(
            # Z goes into a linear of size: ndf
            nn.Linear(nc * isize * isize *isize, ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, 1),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(ndf, 1)
        )
        self.main = main
        self.nc = nc
        self.isize = isize
        self.nz = nz

    def forward(self, input):
        input = input.view(input.size(0),
                           input.size(1) * input.size(2) * input.size(3) *input.size(4))
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        output = output.mean(0)
        return output.view(1)

In [ ]:
#Input minibatches
def plot_mass_hist(fake, real, s_sample, together):
    num_bins=100
    al=0.9
    dens=False
    xlim=.02
    
    plt.figure(figsize=(20,10))
               
    k=0
    for n in  range(fake.shape[0]):
        k+=1
        label='Generated' if k==1 else None
        #plt.hist(np.where(fake[n].reshape(s_sample, s_sample, s_sample).flatten() >0),
        plt.hist(np.where(fake[n].reshape(s_sample, s_sample, s_sample).flatten() > 0),
                alpha=al, bins=num_bins, normed=dens, color='r', label=label)
        
    
    if together==False:
        plt.tick_params(axis='both', which='major', labelsize=18)
        plt.legend(loc='best',fontsize=18)
        plt.xlim(0,xlim)
        plt.show()
        
        plt.figure(figsize=(20,10))
    
    k=0   
    for m in range(real.shape[0]):
        k+=1
        label='Real' if k==1 else None
        plt.hist(np.where(real[m].reshape(s_sample, s_sample, s_sample).flatten() >0),alpha=al,bins=num_bins, 
                 normed=dens ,color='g', label=label)
    
    plt.legend(loc='best',fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.title('PDFs of Hydrogen mass', fontsize=18)
        #plt.savefig('speed_hist.jpg',format='jpg', dpi=100)
    #plt.xlim(0,xlim)
    plt.show()

In [ ]:
#Atakans architecture

In [ ]:
from conv_utils import calculate_conv_output_dim, calculate_pool_output_dim

# input: batch_size * nc * 64 * 64
# output: batch_size * k * 1 * 1
class N_Disc(nn.Module):
    def __init__(self, cube_dimension, fc1_hidden_dim, fc2_output_dim, 
                embedding_dim, leakyrelu_const, pool_return_indices):        
        super(N_Disc, self).__init__()

        self.pool_return_indices = pool_return_indices
      
        # First Convolutional Layer
        self.conv1_in_channels = 1
        self.conv1_out_channels = 2
        self.conv1_kernel = 3
        self.conv1_stride = 1
        self.conv1_padding = 0
        conv1_output_dim = calculate_conv_output_dim(D=cube_dimension,
                                        K=self.conv1_kernel,
                                        P=self.conv1_padding,
                                        S=self.conv1_stride)
        print("Conv1 Output Dimension = " + str(conv1_output_dim))
        self.conv1_encode = nn.Conv3d(in_channels=self.conv1_in_channels, 
                                    out_channels=self.conv1_out_channels, 
                                    kernel_size=self.conv1_kernel, 
                                    stride =self.conv1_stride, 
                                    padding=self.conv1_padding)     
        nn.init.xavier_uniform_(self.conv1_encode.weight)
        self.bn1_encode = nn.BatchNorm3d(num_features = self.conv1_out_channels)
        self.leakyrelu1 = nn.LeakyReLU(leakyrelu_const, inplace=True)

        # First Average Pooling
        self.pool1_kernel = 2
        self.pool1_stride = 2 
        pool1_output_dim = calculate_pool_output_dim(D=conv1_output_dim,
                                                    K=self.pool1_kernel,
                                                    S=self.pool1_stride)
        print("Pool1 Output Dimension = " + str(pool1_output_dim)) 
        self.pool1_encode = nn.MaxPool3d(kernel_size=self.pool1_kernel, 
                                            stride=self.pool1_stride,
                                            return_indices = self.pool_return_indices)

        # Second Convolutional Layer
        self.conv2_in_channels = self.conv1_out_channels
        self.conv2_out_channels = 12
        self.conv2_kernel = 4
        self.conv2_stride = 1
        self.conv2_padding = 0
        conv2_output_dim = calculate_conv_output_dim(D=pool1_output_dim,
                                        K=self.conv2_kernel,
                                        P=self.conv2_padding,
                                        S=self.conv2_stride)
        print("Conv2 Output Dimension= " + str(conv2_output_dim))
        self.conv2_encode = nn.Conv3d(in_channels=self.conv2_in_channels, 
                                    out_channels=self.conv2_out_channels, 
                                    kernel_size=self.conv2_kernel, 
                                    stride =self.conv2_stride, 
                                    padding=self.conv2_padding)     
        nn.init.xavier_uniform_(self.conv2_encode.weight)  
        self.bn2_encode = nn.BatchNorm3d(num_features = self.conv2_out_channels)
        self.leakyrelu2 = nn.LeakyReLU(leakyrelu_const, inplace=True)

        # Second Average Pooling
        self.pool2_kernel = 2
        self.pool2_stride = 2 
        pool2_output_dim = calculate_pool_output_dim(D=conv2_output_dim,
                                                K=self.pool2_kernel,
                                                S=self.pool2_stride)
        print("Pool2 Output Dimension = " + str(pool2_output_dim)) 
        self.pool2_encode = nn.MaxPool3d(kernel_size=self.pool2_kernel, 
                                            stride=self.pool2_stride,
                                            return_indices = self.pool_return_indices)   

        # Third Convolutional Layer
        self.conv3_in_channels = self.conv2_out_channels
        self.conv3_out_channels = 24
        self.conv3_kernel = 3
        self.conv3_stride = 1
        self.conv3_padding = 0
        conv3_output_dim = calculate_conv_output_dim(D=pool2_output_dim,
                                        K=self.conv3_kernel,
                                        P=self.conv3_padding,
                                        S=self.conv3_stride)
        print("Conv3 Output Dimension= " + str(conv3_output_dim))
        self.conv3_encode = nn.Conv3d(in_channels=self.conv3_in_channels, 
                                    out_channels=self.conv3_out_channels, 
                                    kernel_size=self.conv3_kernel, 
                                    stride =self.conv3_stride, 
                                    padding=self.conv3_padding)     
        nn.init.xavier_uniform_(self.conv3_encode.weight)
        self.bn3_encode = nn.BatchNorm3d(num_features = self.conv3_out_channels)
        self.leakyrelu3 = nn.LeakyReLU(leakyrelu_const, inplace=True)  

        # Third Average Pooling
        self.pool3_kernel = 2
        self.pool3_stride = 2 
        pool3_output_dim = calculate_pool_output_dim(D=conv3_output_dim,
                                                K=self.pool3_kernel,
                                                S=self.pool3_stride)
        print("Pool3 Output Dimension = " + str(pool3_output_dim)) 
        self.pool3_encode = nn.MaxPool3d(kernel_size=self.pool3_kernel, 
                                            stride=self.pool3_stride,
                                            return_indices = self.pool_return_indices)  

        # Fourth Convolutional Layer
        self.conv4_in_channels = self.conv3_out_channels
        self.conv4_out_channels = 32
        self.conv4_kernel = 4
        self.conv4_stride = 2
        self.conv4_padding = 0
        conv4_output_dim = calculate_conv_output_dim(D=pool3_output_dim,
                                        K=self.conv4_kernel,
                                        P=self.conv4_padding,
                                        S=self.conv4_stride)
        print("Conv4 Output Dimension= " + str(conv4_output_dim))
        self.conv4_encode = nn.Conv3d(in_channels=self.conv4_in_channels, 
                                    out_channels=self.conv4_out_channels, 
                                    kernel_size=self.conv4_kernel, 
                                    stride =self.conv4_stride, 
                                    padding=self.conv4_padding)     
        nn.init.xavier_uniform_(self.conv4_encode.weight) 
        self.bn4_encode = nn.BatchNorm3d(num_features = self.conv4_out_channels)
        self.leakyrelu4 = nn.LeakyReLU(leakyrelu_const, inplace=True)       

        # Fifth Convolutional Layer
        self.conv5_in_channels = self.conv4_out_channels
        self.conv5_out_channels = 64
        self.conv5_kernel = 3
        self.conv5_stride = 1
        self.conv5_padding = 0
        conv5_output_dim = calculate_conv_output_dim(D=conv4_output_dim,
                                        K=self.conv5_kernel,
                                        P=self.conv5_padding,
                                        S=self.conv5_stride)
        print("Conv5 Output Dimension= " + str(conv5_output_dim))
        self.conv5_encode = nn.Conv3d(in_channels=self.conv5_in_channels, 
                                    out_channels=self.conv5_out_channels, 
                                    kernel_size=self.conv5_kernel, 
                                    stride =self.conv5_stride, 
                                    padding=self.conv5_padding)     
        nn.init.xavier_uniform_(self.conv5_encode.weight) 
        self.bn5_encode = nn.BatchNorm3d(num_features = self.conv5_out_channels)
        self.leakyrelu5 = nn.LeakyReLU(leakyrelu_const, inplace=True)

        # Sixth Convolutional Layer
        self.conv6_in_channels = self.conv5_out_channels
        self.conv6_out_channels = 128
        self.conv6_kernel = 2
        self.conv6_stride = 1
        self.conv6_padding = 0
        conv6_output_dim = calculate_conv_output_dim(D=conv5_output_dim,
                                        K=self.conv6_kernel,
                                        P=self.conv6_padding,
                                        S=self.conv6_stride)
        print("Conv6 Output Dimension= " + str(conv6_output_dim))
        self.conv6_encode = nn.Conv3d(in_channels=self.conv6_in_channels, 
                                    out_channels=self.conv6_out_channels, 
                                    kernel_size=self.conv6_kernel, 
                                    stride =self.conv6_stride, 
                                    padding=self.conv6_padding)     
        nn.init.xavier_uniform_(self.conv6_encode.weight) 
        self.bn6_encode = nn.BatchNorm3d(num_features = self.conv6_out_channels)
        self.leakyrelu6 = nn.LeakyReLU(leakyrelu_const, inplace=True)

        # 7th Convolutional Layer
        self.conv7_in_channels = self.conv6_out_channels
        self.conv7_out_channels = 1
        self.conv7_kernel = 2
        self.conv7_stride = 2
        self.conv7_padding = 0
        conv7_output_dim = calculate_conv_output_dim(D=conv6_output_dim,
                                        K=self.conv7_kernel,
                                        P=self.conv7_padding,
                                        S=self.conv7_stride)
        print("Conv7 Output Dimension= " + str(conv7_output_dim))
        
        self.conv7_encode = nn.Conv3d(in_channels=self.conv7_in_channels, 
                                    out_channels=self.conv7_out_channels, 
                                    kernel_size=self.conv7_kernel, 
                                    stride =self.conv7_stride, 
                                    padding=self.conv7_padding)     
        
        nn.init.xavier_uniform_(self.conv7_encode.weight) 
        #self.bn7_encode = nn.BatchNorm3d(num_features = self.conv7_out_channels)
        
        #self.leakyrelu7 = nn.LeakyReLU(leakyrelu_const, inplace=True)
     
    #         # 1st FC Layer
#         self.fc1_in_features = self.conv7_out_channels * conv7_output_dim**3
#         self.fc1_encode = nn.Linear(in_features=self.fc1_in_features,
#                                     out_features=fc1_hidden_dim)
#         self.leakyrelu8 = nn.LeakyReLU(leakyrelu_const, inplace=True)

#         # 2nd FC Layer
#         self.fc2_encode = nn.Linear(in_features=self.fc1_hidden_dim,
#                                     out_features=embedding_dim)
#         self.relu1 = nn.ReLU(inplace=True)

    def forward(self, input):
        
        # Convolution Layers
#         print("Input = " +str(input.shape))
        out = self.conv1_encode(input)
#         print("conv1_encode = " + str(out.shape))
        out = self.pool1_encode(out)
#         print("pool1_encode = " + str(out.shape))
        out = self.bn1_encode(out) 
        out = self.leakyrelu1(out)

        out = self.conv2_encode(out)
#         print("conv2_encode = " + str(out.shape))
        out = self.pool2_encode(out)
#         print("pool2_encode = " + str(out.shape))
        out = self.bn2_encode(out) 
        out = self.leakyrelu2(out)

        out = self.conv3_encode(out)
#         print("conv3_encode = " + str(out.shape))
        out = self.pool3_encode(out)
#         print("pool3_encode = " + str(out.shape))
        out = self.bn3_encode(out) 
        out = self.leakyrelu3(out)

        out = self.conv4_encode(out)
#         print("conv4_encode = " + str(out.shape))
        out = self.bn4_encode(out) 
        out = self.leakyrelu4(out)

        out = self.conv5_encode(out)
#         print("conv5_encode = " + str(out.shape))
        out = self.bn5_encode(out) 
        out = self.leakyrelu5(out)

        out = self.conv6_encode(out)
#         print("conv6_encode = " + str(out.shape))
        out = self.bn6_encode(out) 
        out = self.leakyrelu6(out)
        
        out = self.conv7_encode(out)
#         print("conv7_encode = " + str(out.shape))
        #out = self.bn7_encode(out) 
        #out = self.leakyrelu7(out)
        
#         print("out = " + str(out.shape))

#         # Transform
        print('D last', out.shape)
        

        #out = out.view(batchSize, -1)

#         # FC Layers
#         out = self.fc1_encode(out)
#         out = self.leakyrelu8(out)

#         out = self.fc2_encode(out)
#         out = self.relu1(out)        
        out= out.mean(0)
        return out.view(1)

In [ ]:
def plot_densities(fake, real, log_):
    fig, ax = plt.subplots(figsize=(20,10))
    ax.plot(xlabel='big', ylabel='big', title='Density')
    sns.set(rc={'figure.figsize':(20,10),"lines.linewidth": 3})
    #plt.subplots(figsize=(20,10))
    #ax.set_ylim(0,0.6)
    #ax.set_xlim(0, )
    bw=0.2
    eps=0.0000000000000001
    eps=0
    grid=200
    n=0
    if log_==True:
        fake=np.log(fake+eps)
        real=np.log(real+eps)
    for m in range(fake.shape[0]):
        n+=1
        if n==1:
            sns.kdeplot(fake[m][0].flatten(), ax=ax, bw=bw, cut=0, label='Generated', color='red', gridsize=grid);
            sns.kdeplot(real[m][0].flatten(), ax=ax, bw=bw, cut=0, label='Real' ,color='blue',gridsize=grid);
        else:
            sns.kdeplot(fake[m][0].flatten(), ax=ax, bw=bw, cut=0, color='red', gridsize=grid);
            sns.kdeplot(real[m][0].flatten(), ax=ax, bw=bw, cut=0,color='blue', gridsize=grid);
        
        ax.set_title("Densities of log HI mass",fontsize=25)
        ax.tick_params(labelsize=20)
        plt.setp(ax.get_legend().get_texts(), fontsize='22')
    plt.show()

In [ ]:
from conv_utils import calculate_deconv_output_dim

# input: batch_size * k * 1 * 1
# output: batch_size * nc * image_size * image_size
class N_Gen(nn.Module):
    def __init__(self, embedded_cube_dimension,fc1_hidden_dim, fc2_output_dim, 
                embedding_dim, leakyrelu_const):
        super(N_Gen, self).__init__()

#         # 1st FC Layer
#         self.embedding_dim = embedding_dim
#         self.fc1_in_features = self.embedding_dim 
#         self.fc1_hidden_dim = fc1_hidden_dim
#         self.fc1_decode = nn.Linear(in_features=self.fc1_in_features,
#                                     out_features=self.fc1_hidden_dim)
#         self.leakyrelu1 = nn.LeakyReLU(leakyrelu_const, inplace=True)

#         # 2nd FC Layer
#         self.fc2_output_dim = fc2_output_dim
#         self.fc2_decode = nn.Linear(in_features=self.fc1_hidden_dim,
#                                     out_features=self.fc2_output_dim )
#         self.leakyrelu2 = nn.LeakyReLU(leakyrelu_const, inplace=True)

        # 1st Deconvolutional Layer
        self.deconv1_in_channels = batch_size
        self.deconv1_out_channels = 128
        self.deconv1_kernel = 2
        self.deconv1_stride = 1
        self.deconv1_padding = 0
        deconv1_output_dim = calculate_deconv_output_dim(D=embedded_cube_dimension,
                                        K=self.deconv1_kernel,
                                        P=self.deconv1_padding,
                                        S=self.deconv1_stride)
        print("Deconv1 Output Dimension = " + str(deconv1_output_dim))
        self.deconv1_decode = nn.ConvTranspose3d(in_channels=self.deconv1_in_channels, 
                                    out_channels=self.deconv1_out_channels, 
                                    kernel_size=self.deconv1_kernel, 
                                    stride =self.deconv1_stride, 
                                    padding=self.deconv1_padding)     
        nn.init.xavier_uniform_(self.deconv1_decode.weight)
        self.bn1_decode = nn.BatchNorm3d(num_features = self.deconv1_out_channels)
        self.leakyrelu1 = nn.LeakyReLU(leakyrelu_const, inplace=True)

        # 2nd Deconvolutional Layer
        self.deconv2_in_channels = self.deconv1_out_channels
        self.deconv2_out_channels = 64
        self.deconv2_kernel = 2
        self.deconv2_stride = 1
        self.deconv2_padding = 0
        deconv2_output_dim = calculate_deconv_output_dim(D=deconv1_output_dim,
                                        K=self.deconv2_kernel,
                                        P=self.deconv2_padding,
                                        S=self.deconv2_stride)
        print("Deconv2 Output Dimension = " + str(deconv2_output_dim))
        self.deconv2_decode = nn.ConvTranspose3d(in_channels=self.deconv2_in_channels, 
                                    out_channels=self.deconv2_out_channels, 
                                    kernel_size=self.deconv2_kernel, 
                                    stride =self.deconv2_stride, 
                                    padding=self.deconv2_padding)     
        nn.init.xavier_uniform_(self.deconv2_decode.weight)
        self.bn2_decode = nn.BatchNorm3d(num_features = self.deconv2_out_channels)
        self.leakyrelu2 = nn.LeakyReLU(leakyrelu_const, inplace=True)
        
        # 3rd Deconvolutional Layer
        self.deconv3_in_channels = self.deconv2_out_channels
        self.deconv3_out_channels = 32
        self.deconv3_kernel = 3
        self.deconv3_stride = 1
        self.deconv3_padding = 0
        deconv3_output_dim = calculate_deconv_output_dim(D=deconv2_output_dim,
                                        K=self.deconv3_kernel,
                                        P=self.deconv3_padding,
                                        S=self.deconv3_stride)
        print("Deconv3 Output Dimension = " + str(deconv3_output_dim))
        self.deconv3_decode = nn.ConvTranspose3d(in_channels=self.deconv3_in_channels, 
                                    out_channels=self.deconv3_out_channels, 
                                    kernel_size=self.deconv3_kernel, 
                                    stride =self.deconv3_stride, 
                                    padding=self.deconv3_padding)     
        nn.init.xavier_uniform_(self.deconv3_decode.weight)
        self.bn3_decode = nn.BatchNorm3d(num_features = self.deconv3_out_channels)
        self.leakyrelu3 = nn.LeakyReLU(leakyrelu_const, inplace=True)
        
        # 4th Deconvolutional Layer
        self.deconv4_in_channels = self.deconv3_out_channels
        self.deconv4_out_channels = 24
        self.deconv4_kernel = 4
        self.deconv4_stride = 2
        self.deconv4_padding = 0
        deconv4_output_dim = calculate_deconv_output_dim(D=deconv3_output_dim,
                                        K=self.deconv4_kernel,
                                        P=self.deconv4_padding,
                                        S=self.deconv4_stride)
        print("Deconv4 Output Dimension = " + str(deconv4_output_dim))
        self.deconv4_decode = nn.ConvTranspose3d(in_channels=self.deconv4_in_channels, 
                                    out_channels=self.deconv4_out_channels, 
                                    kernel_size=self.deconv4_kernel, 
                                    stride =self.deconv4_stride, 
                                    padding=self.deconv4_padding)     
        nn.init.xavier_uniform_(self.deconv4_decode.weight)
        self.bn4_decode = nn.BatchNorm3d(num_features = self.deconv4_out_channels)
        self.leakyrelu4 = nn.LeakyReLU(leakyrelu_const, inplace=True)
        
        # Avg Unpooling 1
        # Just make 1 voxel to 8 voxels of 2-len edges
        # Implemented in forward pass
        self.avgunpool1_scale = 2
        self.avgunpool1 = nn.Upsample(scale_factor = self.avgunpool1_scale, mode='nearest')
        
        
        # 5th Deconvolutional Layer
        self.deconv5_in_channels = self.deconv4_out_channels
        self.deconv5_out_channels = 12
        self.deconv5_kernel = 3
        self.deconv5_stride = 1
        self.deconv5_padding = 0
        deconv5_output_dim = calculate_deconv_output_dim(D=deconv4_output_dim * self.avgunpool1_scale,
                                        K=self.deconv5_kernel,
                                        P=self.deconv5_padding,
                                        S=self.deconv5_stride)
        print("Deconv5 Output Dimension = " + str(deconv5_output_dim))
        self.deconv5_decode = nn.ConvTranspose3d(in_channels=self.deconv5_in_channels, 
                                    out_channels=self.deconv5_out_channels, 
                                    kernel_size=self.deconv5_kernel, 
                                    stride =self.deconv5_stride, 
                                    padding=self.deconv5_padding)     
        nn.init.xavier_uniform_(self.deconv5_decode.weight)
        self.bn5_decode = nn.BatchNorm3d(num_features = self.deconv5_out_channels)
        self.leakyrelu5 = nn.LeakyReLU(leakyrelu_const, inplace=True)
        
        # Avg Unpooling 2
        # Just make 1 voxel to 8 voxels of 2-len edges
        # Implemented in forward pass
        self.avgunpool2_scale = 2
        self.avgunpool2 = nn.Upsample(scale_factor = self.avgunpool2_scale, mode='nearest')
        
        # 6th Deconvolutional Layer
        self.deconv6_in_channels = self.deconv5_out_channels
        self.deconv6_out_channels = 2
        self.deconv6_kernel = 4
        self.deconv6_stride = 1
        self.deconv6_padding = 0
        deconv6_output_dim = calculate_deconv_output_dim(D=deconv5_output_dim * self.avgunpool2_scale,
                                        K=self.deconv6_kernel,
                                        P=self.deconv6_padding,
                                        S=self.deconv6_stride)
        print("Deconv6 Output Dimension = " + str(deconv6_output_dim))
        self.deconv6_decode = nn.ConvTranspose3d(in_channels=self.deconv6_in_channels, 
                                    out_channels=self.deconv6_out_channels, 
                                    kernel_size=self.deconv6_kernel, 
                                    stride =self.deconv6_stride, 
                                    padding=self.deconv6_padding)     
        nn.init.xavier_uniform_(self.deconv6_decode.weight)
        self.bn6_decode = nn.BatchNorm3d(num_features = self.deconv6_out_channels)
        self.leakyrelu6 = nn.LeakyReLU(leakyrelu_const, inplace=True)
        
        # Avg Unpooling 3
        # Just make 1 voxel to 8 voxels of 2-len edges
        # Implemented in forward pass
        self.avgunpool3_scale = 2
        self.avgunpool3 = nn.Upsample(scale_factor = self.avgunpool3_scale, mode='nearest')
        
        # 7th Deconvolutional Layer
        self.deconv7_in_channels = self.deconv6_out_channels
        self.deconv7_out_channels = 1
        self.deconv7_kernel = 3
        self.deconv7_stride = 1
        self.deconv7_padding = 0
        deconv7_output_dim = calculate_deconv_output_dim(D=deconv6_output_dim * self.avgunpool3_scale,
                                        K=self.deconv7_kernel,
                                        P=self.deconv7_padding,
                                        S=self.deconv7_stride)
        print("Deconv7 Output Dimension = " + str(deconv7_output_dim))
        self.deconv7_decode = nn.ConvTranspose3d(in_channels=self.deconv7_in_channels, 
                                    out_channels=self.deconv7_out_channels, 
                                    kernel_size=self.deconv7_kernel, 
                                    stride =self.deconv7_stride, 
                                    padding=self.deconv7_padding)     
        nn.init.xavier_uniform_(self.deconv7_decode.weight)
        self.bn7_decode = nn.BatchNorm3d(num_features = self.deconv7_out_channels)
#         self.leakyrelu7 = nn.LeakyReLU(leakyrelu_const, inplace=True)
        
        # For data in [0,1]
#         self.relu7 = nn.ReLU(inplace=True)     
        # For data in [-1,1]
        self.tanh7 = nn.Tanh()     

    def forward(self, input):
#         print("\nDecoder - Forward Pass")
        
        # Deconvolution Layers
#         print("Input = " +str(input.shape))
        out = self.deconv1_decode(input)
#         print("deconv1_decode = " + str(out.shape))
        out = self.bn1_decode(out)
        out = self.leakyrelu1(out)

        out = self.deconv2_decode(out)
#         print("deconv2_decode = " + str(out.shape))
        out = self.bn2_decode(out)
        out = self.leakyrelu2(out)
        
        out = self.deconv3_decode(out)
#         print("deconv3_decode = " + str(out.shape))
        out = self.bn3_decode(out)
        out = self.leakyrelu3(out)
        
        out = self.deconv4_decode(out)
#         print("deconv4_decode = " + str(out.shape))
        out = self.bn4_decode(out)
        out = self.leakyrelu4(out)
        out = self.avgunpool1(out)
#         print("avgunpool1 = " + str(out.shape))

        out = self.deconv5_decode(out)
#         print("deconv5_decode = " + str(out.shape))
        out = self.bn5_decode(out)
        out = self.leakyrelu5(out)
        out = self.avgunpool2(out)
#         print("avgunpool2 = " + str(out.shape))
        out = self.deconv6_decode(out)
#         print("deconv6_decode = " + str(out.shape))
        out = self.bn6_decode(out)
        out = self.leakyrelu6(out)
        out = self.avgunpool3(out) 
#         print("avgunpool3 = " + str(out.shape))
        
        out = self.deconv7_decode(out)
#         print("deconv7_decode = " + str(out.shape))
        out = self.bn7_decode(out)
#         out = self.leakyrelu7(out)
#         out = self.relu7(out) # for [0,1]
        out = self.tanh7(out) # for [-1,1]
        
#         print("decoder out = " + str(out.shape))
#         # Transformation
#         out = out.view(batch_size, nz, 1,1,1)
        
        return out

In [ ]:
class DCGAN_D__(nn.Module):
    def __init__(self, ngpu, cube_len, leak_value, bias):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.cube_len=cube_len
        self.leak_value=leak_value
        self.bias=bias
        
        self.main = nn.Sequential(
            #1
            nn.Conv3d(in_channels=1, 
                      out_channels=self.cube_len, 
                      kernel_size=4, # == 4( 4,4,4)
                      stride = 2, # == 2
                     padding=1,
                     # padding=0,
                      bias=self.bias),
            nn.BatchNorm3d(self.cube_len),
            nn.LeakyReLU(self.leak_value, inplace=True),
        
            #2
            nn.Conv3d(self.cube_len, self.cube_len*2, kernel_size=4, stride=2, bias=self.bias, padding=(1,1,1)),
            nn.BatchNorm3d(self.cube_len*2),
            nn.LeakyReLU(self.leak_value, inplace=True),
  
            
            #3
            nn.Conv3d(self.cube_len*2, self.cube_len*4, kernel_size=4, stride=2, bias=self.bias, padding=(1,1,1)),
            nn.BatchNorm3d(self.cube_len*4),
            nn.LeakyReLU(self.leak_value, inplace=True),
        
            #4
            nn.Conv3d(self.cube_len*4, self.cube_len*8, kernel_size=4, stride=2, bias=self.bias, padding=(1,1,1)),
            nn.BatchNorm3d(self.cube_len*8),
            nn.LeakyReLU(self.leak_value, inplace=True),
            
            #5
            nn.Conv3d(self.cube_len*8, self.cube_len*16, kernel_size=4, stride=2, bias=self.bias, padding=(1,1,1)),
            nn.BatchNorm3d(self.cube_len* 16),
            nn.LeakyReLU(self.leak_value, inplace=True),
            
            #6
            nn.Conv3d(self.cube_len*16, 1, kernel_size=4, stride=1, bias=self.bias, padding=0)
            
            )    
        
    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main,  input, range(self.ngpu))
        else:
            output = self.main(input)
        #print('D0:',output.size)
        output = output.mean(0)
        #print('D1:',output.size)
        #return output.view(-1, 1).squeeze(1)
        return output.view(1)

# DCGAN

In [ ]:
class DCGAN_G__(nn.Module):
    def __init__(self, ngpu, cube_len, leak_value, bias):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.cube_len=cube_len
        self.leak_value=leak_value
        self.bias=bias
        
        self.main = nn.Sequential(
            #1
            nn.ConvTranspose3d(in_channels=nz, 
                      out_channels=self.cube_len*8, 
                      kernel_size=4, # == 4( 4,4,4)
                      stride = 2, # == 2
                     #padding=(1,1,1), # == 1
                      padding=0,
                      bias=self.bias),
            nn.BatchNorm3d(self.cube_len*8),
            nn.ReLU(True),
            
            #2
            nn.ConvTranspose3d(self.cube_len*8, self.cube_len*4, kernel_size=4, stride=2, bias=self.bias, padding=1),
            nn.BatchNorm3d(self.cube_len*4),
            nn.ReLU(True),
            
            #3
            nn.ConvTranspose3d(self.cube_len*4, self.cube_len*2, kernel_size=4, stride=2, bias=self.bias, padding=1),
            nn.BatchNorm3d(self.cube_len*2),
            nn.ReLU(True),
            
            #4
            nn.ConvTranspose3d(self.cube_len*2, self.cube_len, kernel_size=4, stride=2, bias=self.bias, padding=1),
            nn.BatchNorm3d(self.cube_len),
            nn.ReLU(True),
            
            #5 
            nn.ConvTranspose3d(self.cube_len, 1, kernel_size=4, stride=2, bias=self.bias, padding=1)
            
            )    
        
    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output